In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utilities.OneHotEncoder import OneHotEncoder
from utilities.DataPreProcessor import DataPreProcessor

In [2]:
df_pdb_data_no_seq = pd.read_csv('resources/pdb_data_no_dups.csv')
df_pdb_seq = pd.read_csv('resources/pdb_data_seq.csv')
print("..Read data files..")

..Read data files..


In [3]:
# filtering out only protein structures
df_protein_seq = df_pdb_seq[(df_pdb_seq['macromoleculeType'] == 'Protein')]

In [4]:
# joining two data sets for protein Seq - classification

df_protein_final = df_protein_seq.merge(df_pdb_data_no_seq)

df_protein_seq_final = df_protein_final[['sequence', 'classification']]

df_protein_seq_final.head()

,sequence,classification
0,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,OXYGEN TRANSPORT
1,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,HYDROLASE(O-GLYCOSYL)
2,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,OXYGEN TRANSPORT
3,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,HYDROLASE(O-GLYCOSYL)
4,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,OXYGEN TRANSPORT


In [5]:
# drop na values in the data frame

df_protein_seq_final = df_protein_seq_final.dropna()
print("filtered data with NA values")

filtered data with NA values


In [6]:
##One_hot encoding

one_hot_encoder = OneHotEncoder()

one_hot_encoder.find_unique_amino_acids(df_protein_seq_final['sequence'])
print("Unique Amino Acid Set...")

Unique Amino Acid Set...


In [7]:
## creates an encoded dictionaryfor each Amino Acid
one_hot_encoder.create_one_hot_dict()

data_pre_processor = DataPreProcessor(one_hot_encoder,
                                      df_protein_seq_final['classification']
                                      .value_counts().to_dict())

In [8]:
## delete least preferred sequences
data_pre_processor.del_least_preferred()

df_protein_seq_final_for_modeling = \
    df_protein_seq_final[df_protein_seq_final['classification']
        .isin(data_pre_processor.count_dict_keys_as_list())]

df_protein_seq_final_for_modeling = \
    df_protein_seq_final_for_modeling.reset_index(drop=True)
print("final Shape of the DataSet {}".format(df_protein_seq_final_for_modeling.shape))

final Shape of the DataSet (291355, 2)


In [9]:
X_sequences = df_protein_seq_final_for_modeling['sequence']
y_label = df_protein_seq_final_for_modeling['classification']

In [10]:
## test process sequences

data_pre_processor.process_all_seqs(X_sequences[:20], y_label[:20])
# print("Getting Processed Seqs")
# print(data_pre_processor.get_x())
# print("getting labels")
# print(data_pre_processor.get_y())
print("getting protein categories")
print(data_pre_processor.get_protein_categoeies())

getting protein categories
['HYDROLASE', 'LIGASE', 'OXYGEN TRANSPORT', 'TRANSFERASE/TRANSFERASE INHIBITOR']


### LSTM Modeling

In [11]:
# from model.LSTMModel import LSTMModel
# 
# lstm_model = LSTMModel(None,100)

In [12]:
# print(lstm_model.define_model(100,3))

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_label, test_size=0.20, random_state=42)

In [14]:
# for X,y in data_pre_processor.batch_generator(512):
#     X = X.reshape((1,X.shape[0],X.shape[1]))
#     lstm_model.get_model().fit(X,y,epochs=10)

In [43]:
count=0
for X,y in data_pre_processor.batch_generator(1):
    if count == 8:
        print("Finding truth")
        print(X[0][208])
    if count == 1:
        print("Compare")
        print(X[0][25])
    print(X.shape)
    count += 1

(1, 154, 25)
Compare
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(1, 154, 25)
(1, 154, 25)
(1, 154)
(1, 154)
(1, 154, 25)
(1, 154, 25)
(1, 154, 25)
Finding truth
[0, 0, 0]
(1, 209)
(1, 209, 25)
(1, 209, 25)
(1, 209)
(1, 330)
(1, 330)
(1, 330)
(1, 330)
(1, 330, 25)
(1, 330, 25)
(1, 330)
(1, 330)
